In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -qU bs4 tiktoken openai langchain
!pip install pinecone-client[grpc]
#!pip install pinecone-client
#!pip install protobuf

In [ ]:
import os

chat_api_list = ["Add you API KEY for OPENAI",]
os.environ["OPENAI_API_KEY"] = chat_api_list[0]

In [ ]:
import openai

# initialize openai
#openai.api_key = key  #platform.openai.com

embed_model = "text-embedding-ada-002"


res = openai.embeddings.create(
    input=[
        "Climate Bert Team",
        "there will be several texts in each batch"
    ], model=embed_model
)


In [ ]:
import pinecone

indexes = [     # index configurations here...

    {
        "name": "alldatabases-fact-cheker",
        "api_key": "Ask Authors for API Keys",
        "environment": "us-east4-gcp",
        "info" : "FactChecking"
    },
    {
        "name": "alldatabases-fact-cheker",
        "api_key": "Ask Authors for API Keys",
        "environment": "eu-west1-gcp",
        "info" : "FactChecking"
    }
]

selected_indexes = indexes[0]
final_res = []

index_name = selected_indexes['name']

# Initialize Pinecone connection for the selected index
pinecone.init(
    api_key=selected_indexes["api_key"],
    environment=selected_indexes["environment"],
)

# Check if the index exists; if not, create it
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='dotproduct'
    )

# Connect to the index and get stats
index = pinecone.GRPCIndex(index_name)
index.describe_index_stats()


In [ ]:
selected_indexes = indexes[1]
final_res = []
index_name2 = selected_indexes['name']

# Initialize Pinecone connection for the selected index
pinecone.init(
    api_key=selected_indexes["api_key"],
    environment=selected_indexes["environment"],
)

# Check if the index exists; if not, create it
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name2,
        dimension=len(res['data'][0]['embedding']),
        metric='dotproduct'
    )

# Connect to the index and get stats
index2 = pinecone.GRPCIndex(index_name2)
index2.describe_index_stats()

In [ ]:
def get_augmented_query(results, org_name, query, threshold=0.75):
    contexts = []
    pages = []
    filenames = []
    urls = []
    orgs = []

    for res in results:
        for item in res['matches']:
            if item['score'] >= threshold:
                contexts.append(item['metadata']['text'])
                pages.append(item['metadata'].get('page', 'N/A'))  # Default to 'N/A' if 'page' does not exist
                filenames.append(item['metadata'].get('filename', 'N/A'))  # Default to 'N/A' if 'filename' does not exist
                urls.append(item['metadata'].get('url', 'N/A'))  # Default to 'N/A' if 'url' does not exist
                orgs.append(item['metadata'].get('org', 'N/A'))

    user_info = []
    for i in range(len(contexts)):
        user_info.append(str(contexts[i]) + "\nReference:" + str(filenames[i]) + "\nPage:" + str(pages[i]) + "\nORG:" + str(orgs[i]) + "\nURL:" + str(urls[i]) + "\n\n---------------\n\n")

    augmented_query = "\n".join(user_info)

    return augmented_query

### Call our API

In [ ]:
Q1= "Is it still possible to limit warming to 1.5°C?"
Q2= "When will we reach 1.5°C?"
Q3= "What does overshoot mean?"
Q4= "Can we avoid overshooting 1.5°C?"
Q5= "Have emissions reductions fallen for some countries?"
Q6= "What are the issues with financing adaptation?"
Q7= "Where is the majority of climate finance going?"
Q8= "What are the options for scaling up adaptation and mitigation in developing countries?"
Q9= "Which regions will be disproportionally affected by climate change?"
Q10= "What is climate justice?"
Q11= "What is maladaptation?"
Q12= "Is there evidence of maladaptation?"
Q13= "Will glaciers in Scotland melt?"

In [ ]:
### K is a hyperparameter, try k=10, 20, ...

In [ ]:
user_query = Q10

res = openai.embeddings.create(
  input=[user_query],
  model=embed_model
)

final_res_WMO = []
final_res_IPCC = []
final_res_IPCCSYR = []


# We query from the selected index and store the results in a temporary variable and then make augmented query
xq = res.data[0].embedding

temp_res_WMO = index2.query(xq,
                      filter={"org": "WMO"},
                      top_k=5,
                      include_metadata=True)
final_res_WMO.append(temp_res_WMO)

temp_res_IPCC = index.query(xq,
                      filter={"org": "IPCC"},
                      top_k=5,
                      include_metadata=True)
final_res_IPCC.append(temp_res_IPCC)



temp_res_IPCCSYR = index2.query(xq,
                      filter={"org": "IPCC-SYR"},
                      top_k=5,
                      include_metadata=True)
final_res_IPCCSYR.append(temp_res_IPCCSYR)





augmented_query_WMO = get_augmented_query(final_res_WMO, 'WMO', user_query)
augmented_query_IPCC = get_augmented_query(final_res_IPCC, 'IPCC', user_query)
augmented_query_IPCCSYR = get_augmented_query(final_res_IPCCSYR, 'IPCC-SYR', user_query)




### Hybrid ChatClimate / ChatBiodiversity (please modify the primer)

In [ ]:
primer = f"""You are a Q&A bot, an intelligent system that answers user questions based on the information provided by
the user above the question and your in-house knowledge. There are five pieces of extra information above the user question.
The user question is in the final line. When you use the user information, always indicate the Page and Reference in your answer.
The Page and Reference are located below each piece of information. Additionally, let us know which part of your answer is from the IPCC
information and which part is based on your in-house knowledge by writing either (IPCC AR6) or (In-house knowledge).
If the information cannot be found in the information provided by the user or your in-house knowledge, please say 'There is not enough information'."""

res_IPCCSYR = openai.chat.completions.create(

    model="gpt-4",
    temperature = 0,
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query_IPCCSYR + "\n\n" + user_query}
    ]
)

res_IPCCALL = openai.chat.completions.create(
    model="gpt-4",
    temperature = 0,
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query_IPCC + "\n\n" + user_query}
    ]
)

res_WMO = openai.chat.completions.create(
    model="gpt-4",
    temperature = 0,
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query_WMO + "\n\n" + user_query}
    ]
)


In [ ]:
from IPython.display import Markdown

display(Markdown(res_IPCCSYR.choices[0].message.content))


In [ ]:
display(Markdown(res_IPCCALL.choices[0].message.content))

In [ ]:
display(Markdown(res_WMO.choices[0].message.content))

###Standalone chatclimate

In [ ]:

primer = f"""You are a Q&A bot, an intelligent system that answers user questions ONLY based on the information provided by
the user above the question and NEVER USE your in-house knowledge. There are five pieces of extra information above the user question.
The user question is in the final line. When you use the user information, TRY to USE as much as information you can in your answer. We need a comprehensive answer. Always indicate the Page and Reference in your answer.
The Page and Reference are located below each piece of information.
If the information cannot be found in the information provided by the user, please say 'There is not enough information'."""

res_IPCCSYR = openai.chat.completions.create(
    model="gpt-4",
    temperature = 0,
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query_IPCCSYR + "\n\n" + user_query}
    ]
)

res_IPCCALL = openai.chat.completions.create(
    model="gpt-4",
    temperature = 0,
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query_IPCC + "\n\n" + user_query}
    ]
)

res_WMO = openai.chat.completions.create(
    model="gpt-4",
    temperature = 0,
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query_WMO + "\n\n" + user_query}
    ]
)


In [ ]:
display(Markdown(res_IPCCSYR.choices[0].message.content))

In [ ]:
display(Markdown(res_IPCCALL.choices[0].message.content))

In [ ]:
display(Markdown(res_WMO.choices[0].message.content))

### GPT-4

In [ ]:
res = openai.chat.completions.create(
    model="gpt-4",
    temperature = 0,
    messages=[
        {"role": "system", "content": "You are Q&A bot. A highly intelligent system that answers user questions"},
        {"role": "user", "content": user_query}
    ]
)
display(Markdown(res.choices[0].message.content))